In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [3]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [17]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)
print(y_pred)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 0s 32us/sample - loss: 0.8115 - val_loss: 1.6010
Epoch 2/20
11610/11610 [==============================] - 0s 26us/sample - loss: 1.4180 - val_loss: 0.8320
Epoch 3/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4714 - val_loss: 0.4269
Epoch 4/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4121 - val_loss: 0.4054
Epoch 5/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.3973 - val_loss: 0.3917
Epoch 6/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.3867 - val_loss: 0.3975
Epoch 7/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.3805 - val_loss: 0.3906
Epoch 8/20
11610/11610 [==============================] - 0s 28us/sample - loss: 0.3776 - val_loss: 0.3733
Epoch 9/20
11610/11610 [==============================] - 0s 27us/sample - loss: 0.3738 - val_l

In [24]:
# Wide and deep neural net.
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model2 = keras.Model(inputs=[input_], outputs=[output])
model2.compile(loss="mean_squared_error", optimizer="Adam")
print(model2.summary())
history2 = model2.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test2 = model2.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model2.predict(X_new)
print(y_pred)

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_51 (Dense)                (None, 30)           270         input_12[0][0]                   
__________________________________________________________________________________________________
dense_52 (Dense)                (None, 30)           930         dense_51[0][0]                   
__________________________________________________________________________________________________
concatenate_12 (Concatenate)    (None, 38)           0           input_12[0][0]                   
                                                                 dense_52[0][0]            

In [19]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 0s 38us/sample - loss: 2.7266 - val_loss: 1.0831
Epoch 2/20
11610/11610 [==============================] - 0s 31us/sample - loss: 0.8364 - val_loss: 0.7278
Epoch 3/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.6672 - val_loss: 0.6594
Epoch 4/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.6148 - val_loss: 0.6219
Epoch 5/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5830 - val_loss: 0.5929
Epoch 6/20
11610/11610 [==============================] - 0s 28us/sample - loss: 0.5590 - val_loss: 0.5704
Epoch 7/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5400 - val_loss: 0.5514
Epoch 8/20
11610/11610 [==============================] - 0s 28us/sample - loss: 0.5240 - val_loss: 0.5356
Epoch 9/20
11610/11610 [==============================] - 0s 28us/sample - loss: 0.5102 - val_l

In [4]:
model.layers

In [5]:
model.layers[1]

In [7]:
hidden1 = model.layers[1]

In [9]:
hidden1.name

'dense_1'

In [11]:
model.get_layer('dense_1') is hidden1

True